In [23]:
import pandas as pd
import requests as r
import sqlalchemy
import time
import datetime
pd.options.display.max_columns = None
from sqlalchemy import create_engine
from kaggle.api.kaggle_api_extended import KaggleApi

In [32]:
#api command
api = KaggleApi()
api.authenticate()

In [33]:
#obtained from kaggle dataset website
!kaggle datasets download -d bumjunkoo/us-vaccination-progress


  0%|          | 0.00/224k [00:00<?, ?B/s]
100%|##########| 224k/224k [00:00<00:00, 3.59MB/s]


In [34]:
#unzip the *zip file and unzip it in the same directory where is downloaded
!unzip us-vaccination-progress.zip

Archive:  us-vaccination-progress.zip
  inflating: us_state_vaccinations.csv  


In [35]:
#move the unzipped file to whatever folder we want, in this case the "resources" folder
!mv us_state_vaccinations.csv Resources/us_state_vaccinations.csv   

In [37]:
#read the csv files
state_df = pd.read_csv("Resources/us_state_population.csv")
vaccination_df =pd.read_csv("Resources/us_state_vaccinations.csv")

In [38]:
vaccination_df.sample(20)

vaccination_df.dropna(how="all")

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,2021-04-11,Wyoming,310702.0,450525.0,180223.0,22.80,53.68,131933.0,31.14,77.84,21362.0,3431.0,5928.0,0.690
6129,2021-04-12,Wyoming,310765.0,450525.0,180275.0,22.81,53.70,131994.0,31.15,77.84,63.0,3400.0,5875.0,0.690
6130,2021-04-13,Wyoming,314169.0,453025.0,181974.0,23.18,54.28,134166.0,31.44,78.28,3404.0,3700.0,6393.0,0.693
6131,2021-04-14,Wyoming,317111.0,460205.0,183741.0,23.45,54.79,135695.0,31.75,79.52,2942.0,4075.0,7041.0,0.689


In [39]:
#Cleaning the dataframe
vaccination_df=vaccination_df.dropna()
vaccination_df.sample(10)

#this splits dates into days and months for useful filtering and group by function.
vaccination_df["date"]=pd.to_datetime(vaccination_df["date"])
vaccination_df["day"] = vaccination_df["date"].dt.day
vaccination_df["month"] = vaccination_df["date"].dt.month




In [41]:

vaccination_df.sample(10)


,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used,day,month
3546,2021-03-21,New Hampshire,529948.0,651805.0,377145.0,13.60,38.98,184960.0,27.74,47.94,15526.0,9893.0,7276.0,0.813,21,3
3585,2021-01-25,New Jersey,563141.0,989900.0,492061.0,0.78,6.34,69374.0,5.54,11.14,27516.0,26252.0,2956.0,0.569,25,1
5399,2021-01-30,Vermont,63777.0,99750.0,48473.0,2.44,10.22,15213.0,7.77,15.99,2040.0,2050.0,3285.0,0.639,30,1
4360,2021-02-17,Oregon,692781.0,878600.0,487286.0,4.81,16.43,202959.0,11.55,20.83,7567.0,14821.0,3514.0,0.789,17,2
2603,2021-03-18,Marshall Islands,17112.0,35600.0,11597.0,9.35,29.29,5464.0,19.85,60.95,0.0,228.0,3903.0,0.481,18,3
5435,2021-03-07,Vermont,191529.0,261100.0,122891.0,10.33,30.69,64474.0,19.69,41.84,7202.0,4139.0,6633.0,0.734,7,3
2499,2021-03-08,Maine,427445.0,503580.0,287695.0,11.06,31.80,148712.0,21.40,37.46,27911.0,13348.0,9930.0,0.849,8,3
3271,2021-03-28,Montana,499029.0,632625.0,316057.0,18.03,46.69,192701.0,29.57,59.19,11808.0,8000.0,7485.0,0.789,28,3
6010,2021-03-18,Wisconsin,2210128.0,2548695.0,1405196.0,13.79,37.96,802989.0,24.13,43.77,49180.0,46890.0,8053.0,0.867,18,3
936,2021-04-12,Delaware,582422.0,729185.0,373330.0,21.81,59.81,212392.0,38.34,74.88,6310.0,9427.0,9681.0,0.799,12,4


In [43]:
#information to understand if new data has been uploaded and starting month.
month_df=vaccination_df.groupby('month')['day'].min()
month_df

month
1    12
2     1
3     1
4     1
Name: day, dtype: int64

In [44]:
#information to understand if new data has been uploaded and starting day of the month.
vaccination_df.groupby('month')['day'].max()

month
1    31
2    28
3    31
4    15
Name: day, dtype: int64

In [45]:
vaccination_df

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used,day,month
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222,13,1
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226,15,1
7,2021-01-19,Alabama,130795.0,444650.0,114319.0,0.33,2.67,16346.0,2.33,9.07,7557.0,7523.0,1534.0,0.294,19,1
8,2021-01-20,Alabama,139200.0,483275.0,121113.0,0.37,2.84,17956.0,2.47,9.86,8405.0,7880.0,1607.0,0.288,20,1
9,2021-01-21,Alabama,165919.0,493125.0,144429.0,0.44,3.38,21345.0,2.95,10.06,26719.0,10517.0,2145.0,0.336,21,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,2021-04-11,Wyoming,310702.0,450525.0,180223.0,22.80,53.68,131933.0,31.14,77.84,21362.0,3431.0,5928.0,0.690,11,4
6129,2021-04-12,Wyoming,310765.0,450525.0,180275.0,22.81,53.70,131994.0,31.15,77.84,63.0,3400.0,5875.0,0.690,12,4
6130,2021-04-13,Wyoming,314169.0,453025.0,181974.0,23.18,54.28,134166.0,31.44,78.28,3404.0,3700.0,6393.0,0.693,13,4
6131,2021-04-14,Wyoming,317111.0,460205.0,183741.0,23.45,54.79,135695.0,31.75,79.52,2942.0,4075.0,7041.0,0.689,14,4


In [46]:
#grouping by state and month (double group by)
first_day=vaccination_df.groupby(["location","month"]).min()

In [47]:
last_day=vaccination_df.groupby(["location","month"]).max()

In [48]:
monthly_df=last_day-first_day
people_vaccinated= monthly_df['people_vaccinated']
people_vaccinated=pd.DataFrame(people_vaccinated)
people_vaccinated

people_vaccinated
location  month                   
Alabama   1               223509.0
          2               348117.0
          3               501830.0
          4               257767.0
Alaska    1                70348.0
...                            ...
Wisconsin 4               470688.0
Wyoming   1                33233.0
          2                51694.0
          3                57030.0
          4                22844.0

[243 rows x 1 columns]

In [49]:
state_df

,state,name,popestimate2019
0,1,Alabama,4903185
1,2,Alaska,731545
2,4,Arizona,7278717
3,5,Arkansas,3017804
4,6,California,39512223
5,8,Colorado,5758736
6,9,Connecticut,3565287
7,10,Delaware,973764
8,11,District of Columbia,705749
9,12,Florida,21477737


In [50]:
state_df=state_df.rename(columns={'name':'location'})
state_df

,state,location,popestimate2019
0,1,Alabama,4903185
1,2,Alaska,731545
2,4,Arizona,7278717
3,5,Arkansas,3017804
4,6,California,39512223
5,8,Colorado,5758736
6,9,Connecticut,3565287
7,10,Delaware,973764
8,11,District of Columbia,705749
9,12,Florida,21477737


In [51]:
merged_df=people_vaccinated.merge(state_df, on='location')
merged_df

,location,people_vaccinated,state,popestimate2019
0,Alabama,223509.0,1,4903185
1,Alabama,348117.0,1,4903185
2,Alabama,501830.0,1,4903185
3,Alabama,257767.0,1,4903185
4,Alaska,70348.0,2,731545
...,...,...,...,...
199,Wisconsin,470688.0,55,5822434
200,Wyoming,33233.0,56,578759
201,Wyoming,51694.0,56,578759
202,Wyoming,57030.0,56,578759


In [52]:
merged_df=merged_df.rename(columns={'popestimate2019':'population'})
merged_df['per_month_people_vac_p_month']=(merged_df['people_vaccinated']/merged_df['population'])*100

merged_df

,location,people_vaccinated,state,population,per_month_people_vac_p_month
0,Alabama,223509.0,1,4903185,4.558445
1,Alabama,348117.0,1,4903185,7.099814
2,Alabama,501830.0,1,4903185,10.234776
3,Alabama,257767.0,1,4903185,5.257134
4,Alaska,70348.0,2,731545,9.616360
...,...,...,...,...,...
199,Wisconsin,470688.0,55,5822434,8.084042
200,Wyoming,33233.0,56,578759,5.742114
201,Wyoming,51694.0,56,578759,8.931870
202,Wyoming,57030.0,56,578759,9.853842


In [53]:
#creating an engine for database. See vaccines1.db is the name of the newly created database to store data.
engine=sqlalchemy.create_engine('sqlite:///vaccines1.db')

In [56]:
#Boiler plate code to determine if new data has been added to the dataset. It will update the tables in the database
#with the append command. Note that "name" and "table" are standard code lines. Upate with table name. Selects refers to
#the column names as learned in SQL

table_exists = engine.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='monthly_vacc';")

if len([x for x in table_exists])>0:
    print('we have a table')
    d= engine.execute("SELECT location,people_vaccinated FROM monthly_vacc")
    loc=[]
    people=[]
    
    for i in d:
        loc.append(i[0])
        people.append(i[1])
    rows_to_insert = merged_df.loc[ ~(
                (merged_df['location'].isin(loc)) 
                & (merged_df['people_vaccinated'].isin(people))
               )]
    
       
   
    if not rows_to_insert.empty:
        print('we have some new rows')
        merged_df.to_sql("monthly_vacc", index=False, if_exists="append" ,con=engine)
        
else:
    merged_df.to_sql("monthly_vacc", index=False, con=engine)

we have a table


In [55]:
#reads the database using pandas. This pulls from the db and not from the dataframe
pd.read_sql(''' select location,per_month_people_vac_p_month from monthly_vacc;''',con=engine)

,location,per_month_people_vac_p_month
0,Alabama,4.558445
1,Alabama,7.099814
2,Alabama,10.234776
3,Alabama,2.186395
4,Alaska,9.616360
...,...,...
403,Wisconsin,8.084042
404,Wyoming,5.742114
405,Wyoming,8.931870
406,Wyoming,9.853842
